## Setup

In [1]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch

#mpl.rcParams['figure.figsize'] = (8, 6)
#mpl.rcParams['axes.grid'] = False

In [14]:
# Reading/Creating the paths to Figshare dataset
path = r'./data/'
#path = r'/home/ubuntu/MIE1075/data/'
dir = []
for filename in os.listdir(path):

    if os.path.isfile(os.path.join(path, filename)):
        if 'walking_' in filename or '_stairup_' in filename or '_stairdown' in filename:
            dir.append(os.path.join(path, filename))
print(len(dir))
dir

22


['./data/preliminary_testing_walking_18_0_2022_12_12-20_19_01.csv',
 './data/preliminary_testing_walking_18_15_2022_12_12-20_24_50.csv',
 './data/preliminary_testing_walking_08_15_2022_12_12-20_22_39.csv',
 './data/preliminary_testing_walking_17_0_2022_12_12-20_18_30.csv',
 './data/preliminary_testing_walking_15_0_2022_12_12-20_16_35.csv',
 './data/preliminary_testing_walking_15_15_2022_12_12-20_24_17.csv',
 './data/preliminary_testing_walking_16_0_2022_12_12-20_20_55.csv',
 './data/preliminary_testing_walking_07_15_2022_12_12-20_25_23.csv',
 './data/preliminary_testing_walking_11_0_2022_12_12-20_20_26.csv',
 './data/preliminary_testing_stairup_0_0_2022_12_12-20_34_19.csv',
 './data/preliminary_testing_walking_05_15_2022_12_12-20_22_04.csv',
 './data/preliminary_testing_walking_08_0_2022_12_12-20_15_29.csv',
 './data/preliminary_testing_walking_10_0_2022_12_12-20_16_02.csv',
 './data/preliminary_testing_walking_10_15_2022_12_12-20_23_18.csv',
 './data/preliminary_testing_walking_13_15_

In [15]:
columns_tofilter = ['position_1','position_2','position_3','position_4', 'velocity_1', 'velocity_2','velocity_3','velocity_4','att_w','att_x','att_y','att_z','rate_dps_x',
                        'rate_dps_y','rate_dps_z','accel_mps2_x','accel_mps2_y','accel_mps2_z','euler_rad_x','euler_rad_y','euler_rad_z']

In [16]:
from scipy import signal

# function for butterworth '
def butter_filt(signal_data):
    fs = 38
    fc = 8  # Cut-off frequency of the filter
    w = fc / (fs / 2) # Normalize the frequency
    b, a = signal.butter(6, w/2, 'lowpass')
    return signal.filtfilt(b, a, signal_data)


In [17]:
def df_to_datapoints(df, num_input=40, prediction_offset=1,stride =38,  output_column=None, filter=True):
    x_list = []
    y_list = []
    df = df.drop(['timestamp'], axis=1)
    if filter == True:
        df[columns_tofilter] = df[columns_tofilter].apply(butter_filt)

    for i in range(((len(df) - num_input - prediction_offset)//stride)-1, stride):
        x = df[i:i+num_input]
        y = df.iloc[[i + num_input + prediction_offset]]


        if output_column is not None:
            y = y[output_column]

        x_list.append(x.values)
        y_list.append(y.values)

    return x_list, y_list

#sample_df = pd.read_csv('/home/ubuntu/MIE1075/data/preliminary_testing_normalwalk_0_0_2022_12_12-20_53_28.csv')
#df_to_datapoints(sample_df, output_column=['position_1', 'velocity_1','position_3', 'velocity_3'])

In [20]:
# intentions are 1 = LEVEL GROUND WALKING | 2 = RAMP UP | 3 = STAIRS UP | 4 = STAIRS DOWN
# Reading the dataset


x_train_all = []
y_train_all = []
x_val_all = []
y_val_all = []
'''
x_walk = []
y_walk = []
x_ramp = []
y_ramp = []
x_stairup = []
y_stairup = []
x_stairdown = []
y_stairdown = []
'''
for f in dir:


    if 'walking_' in f and '_0_' in f:

        temp = pd.read_csv(f, index_col=None, header=0)
        temp['intent'] = 0


    elif 'walking_' in f and '_15_2022' in f:
        print(f)
        temp = pd.read_csv(f, index_col=None, header=0)
        temp['intent'] = 1

    x, y = df_to_datapoints(temp, num_input=40, prediction_offset=1,stride=38, output_column=['position_1', 'velocity_1','position_3', 'velocity_3'])
    n = len(x)
    x_train = x[0:int(n*0.8)]
    y_train = y[0:int(n*0.8)]
    x_val = x[int(n*0.8):]
    y_val = y[int(n*0.8):]
    x_train_all += x_train
    y_train_all += y_train
    x_val_all += x_val
    y_val_all += y_val


    # https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison

    #df_list.append(temp)
#df = pd.concat(df_list, axis=0, ignore_index=True)
#df.to_csv(path+'merged_recorded_data.csv')
print(len(x_train_all),len(y_train_all),len(x_val_all),len(y_val_all))
print(x_train_all[0].shape, y_train_all[0].shape,x_val_all[0].shape, y_val_all[0].shape)

./data/preliminary_testing_walking_18_15_2022_12_12-20_24_50.csv
./data/preliminary_testing_walking_08_15_2022_12_12-20_22_39.csv
./data/preliminary_testing_walking_15_15_2022_12_12-20_24_17.csv
./data/preliminary_testing_walking_07_15_2022_12_12-20_25_23.csv
./data/preliminary_testing_walking_05_15_2022_12_12-20_22_04.csv
./data/preliminary_testing_walking_10_15_2022_12_12-20_23_18.csv
./data/preliminary_testing_walking_13_15_2022_12_12-20_23_49.csv
352 352 89 89
(40, 22) (1, 4) (40, 22) (1, 4)


In [30]:
# intentions are 1 = LEVEL GROUND WALKING | 2 = RAMP UP | 3 = STAIRS UP | 4 = STAIRS DOWN
# Reading the dataset


x_train_all = []
y_train_all = []
x_val_all = []
y_val_all = []
'''
x_walk = []
y_walk = []
x_ramp = []
y_ramp = []
x_stairup = []
y_stairup = []
x_stairdown = []
y_stairdown = []
'''
for f in dir:



    if 'walking_' in f:
        print(f)
        temp = pd.read_csv(f, index_col=None, header=0)
        if '_15_2022' in f:
            temp['intent'] = 1
            print('here')
        elif '_0_' in f:
            temp['intent'] = 0
            print('hreh hrehrhe')
        else:
            pass



    x, y = df_to_datapoints(temp, num_input=40, prediction_offset=1,stride=38, output_column=['position_1', 'velocity_1','position_3', 'velocity_3'])
    n = len(x)
    print(len(x))
    x_train = x[0:int(n*0.8)]
    y_train = y[0:int(n*0.8)]
    x_val = x[int(n*0.8):]
    y_val = y[int(n*0.8):]
    x_train_all += x_train
    y_train_all += y_train
    x_val_all += x_val
    y_val_all += y_val


    # https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison

    #df_list.append(temp)
#df = pd.concat(df_list, axis=0, ignore_index=True)
#df.to_csv(path+'merged_recorded_data.csv')
print(len(x_train_all),len(y_train_all),len(x_val_all),len(y_val_all))
print(x_train_all[0].shape, y_train_all[0].shape,x_val_all[0].shape, y_val_all[0].shape)

./data/preliminary_testing_walking_18_0_2022_12_12-20_19_01.csv
hreh hrehrhe
20
./data/preliminary_testing_walking_18_15_2022_12_12-20_24_50.csv
here
20
./data/preliminary_testing_walking_08_15_2022_12_12-20_22_39.csv
here
20
./data/preliminary_testing_walking_17_0_2022_12_12-20_18_30.csv
hreh hrehrhe
20
./data/preliminary_testing_walking_15_0_2022_12_12-20_16_35.csv
hreh hrehrhe
20
./data/preliminary_testing_walking_15_15_2022_12_12-20_24_17.csv
here
20
./data/preliminary_testing_walking_16_0_2022_12_12-20_20_55.csv
hreh hrehrhe
20
./data/preliminary_testing_walking_07_15_2022_12_12-20_25_23.csv
here
20
./data/preliminary_testing_walking_11_0_2022_12_12-20_20_26.csv
hreh hrehrhe
20
20
./data/preliminary_testing_walking_05_15_2022_12_12-20_22_04.csv
here
20
./data/preliminary_testing_walking_08_0_2022_12_12-20_15_29.csv
hreh hrehrhe
20
./data/preliminary_testing_walking_10_0_2022_12_12-20_16_02.csv
hreh hrehrhe
20
./data/preliminary_testing_walking_10_15_2022_12_12-20_23_18.csv
here
20

In [ ]:
print(np.shape(x_train_all), np.shape(y_train_all))

In [31]:
18*20

360

In [ ]:
print(np.shape(x_val_all), np.shape(y_val_all))

In [ ]:
print(np.shape(x_train_all[0]))

In [ ]:
y_train_all[0]

In [ ]:
from sklearn.preprocessing import StandardScaler
# write a for loop to do over all the data
scalar = StandardScaler()
x_train_all_std = []
for i in x_train_all:
    temp = scalar.fit_transform(i)
    x_train_all_std.append(temp)


In [ ]:
print(np.shape(x_train_all_std[0]))
len(x_train_all_std)


In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler

def get_scaler(scaler):
    scalers = {
        "minmax": MinMaxScaler,
        "standard": StandardScaler,
        "maxabs": MaxAbsScaler,
        "robust": RobustScaler,
    }
    return scalers.get(scaler.lower())()

scaler = get_scaler('robust')

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 16
batch_size_val = 1
torch.manual_seed(101)
train_features = torch.Tensor(x_train_all)
train_targets = torch.Tensor(y_train_all)
val_features = torch.Tensor(x_val_all)
val_targets = torch.Tensor(y_val_all)

train = TensorDataset(train_features, train_targets)
val = TensorDataset(val_features, val_targets)

train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val, batch_size=batch_size_val, shuffle=False, drop_last=True)


In [ ]:
a, b = next(iter(val_loader))
b.shape

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(RNNModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # RNN layers
        self.rnn = nn.RNN(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        out, h0 = self.rnn(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)
        return out

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(LSTMModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # LSTM layers
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Initializing cell state for first input with zeros
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        # Forward propagation by passing in the input, hidden state, and cell state into the model
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)

        return out

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(GRUModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.layer_dim = layer_dim
        self.hidden_dim = hidden_dim

        # GRU layers
        self.gru = nn.GRU(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        out, _ = self.gru(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)

        return out

In [ ]:
def get_model(model, model_params):
    models = {
        "rnn": RNNModel,
        "lstm": LSTMModel,
        "gru": GRUModel,
    }
    return models.get(model.lower())(**model_params)

In [ ]:
class Optimization:
    def __init__(self, model, loss_fn, optimizer):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_losses = []
        self.val_losses = []

    def train_step(self, x, y):
        # Sets model to train mode
        self.model.train()

        # Makes predictions
        yhat = self.model(x)
        #print('shape of y_hat is: ',yhat.shape)
        #print('shape of y is: ',y.shape)
        # Computes loss
        loss = self.loss_fn(y, yhat)

        # Computes gradients
        loss.backward()

        # Updates parameters and zeroes gradients
        self.optimizer.step()
        self.optimizer.zero_grad()

        # Returns the loss
        return loss.item()

In [ ]:
from datetime import datetime
device = torch.device("cpu")
def train(self, train_loader, val_loader, batch_size=64, n_epochs=50, n_features=1):
        model_path = f'models/{self.model}_{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'

        for epoch in range(1, n_epochs + 1):
            batch_losses = []
            for x_batch, y_batch in train_loader:
                x_batch = x_batch.view([batch_size, -1, n_features]).to(device)
                #y_batch = y_batch.view([batch_size, -1, 4]).to(device)
                y_batch = y_batch.reshape(batch_size,-1).to(device)
                #print(x_batch.shape,y_batch.shape)
                loss = self.train_step(x_batch, y_batch)
                batch_losses.append(loss)
            training_loss = np.mean(batch_losses)
            self.train_losses.append(training_loss)

            with torch.no_grad():
                batch_val_losses = []
                for x_val, y_val in val_loader:
                    x_val = x_val.view([batch_size, -1, n_features]).to(device)
                    #y_val = y_val.view([batch_size, -1, 4]).to(device)
                    y_val = y_val.reshape(batch_size,-1).to(device)
                    self.model.eval()
                    yhat = self.model(x_val)
                    val_loss = self.loss_fn(y_val, yhat).item()
                    batch_val_losses.append(val_loss)
                validation_loss = np.mean(batch_val_losses)
                self.val_losses.append(validation_loss)

            if (epoch <= 10) | (epoch % 50 == 0):
                print(
                    f"[{epoch}/{n_epochs}] Training loss: {training_loss:.4f}\t Validation loss: {validation_loss:.4f}"
                )

        torch.save(self.model.state_dict(), model_path)

Optimization.train = train

In [ ]:
for x_val, y_val in val_loader:
    continue
print(x_val.shape,y_val.shape, y_val.view([batch_size, -1, 4]).shape)

In [ ]:
for x_train, y_train in train_loader:
    break
print(x_train.shape,y_train.shape, y_train.view([batch_size, -1, 4]).shape)

In [ ]:
def evaluate(self, test_loader, batch_size=1, n_features=1):
        with torch.no_grad():
            predictions = []
            values = []
            for x_test, y_test in test_loader:

                x_test = x_test.view([batch_size, -1, n_features]).to(device)
                y_test = y_test.reshape(batch_size,-1).to(device)
                #print('x_test and y_test shapes are : ',x_test.shape, y_test.shape)
                self.model.eval()
                yhat = self.model(x_test)
                print(yhat.shape)
                print()
                predictions.append(yhat.to(device).detach().numpy())
                values.append(y_test.to(device).detach().numpy())

        return predictions, values

Optimization.evaluate = evaluate

In [ ]:
 def plot_losses(self):
    plt.plot(self.train_losses, label="Training loss")
    plt.plot(self.val_losses, label="Validation loss")
    plt.legend()
    plt.title("Losses")
    plt.show()
    plt.close()

Optimization.plot_losses = plot_losses

In [ ]:
import torch.optim as optim

input_dim = 22
output_dim = 4
hidden_dim = 8
layer_dim = 3
batch_size = 16
dropout = 0.1
n_epochs = 1000
learning_rate = 1e-1
weight_decay = 1e-6

model_params = {'input_dim': input_dim,
                'hidden_dim' : hidden_dim,
                'layer_dim' : layer_dim,
                'output_dim' : output_dim,
                'dropout_prob' : dropout}

model = get_model('lstm', model_params)

loss_fn = nn.MSELoss(reduction="mean")
#optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
optimizer = optim.SGD(model.parameters(), lr =learning_rate, momentum=0.2, weight_decay=weight_decay)
opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer)
opt.train(train_loader, val_loader, batch_size=batch_size, n_epochs=n_epochs, n_features=input_dim)
opt.plot_losses()


In [ ]:
predictions, values = opt.evaluate(val_loader, batch_size=1, n_features=input_dim)


In [356]:
model

LSTMModel(
  (lstm): LSTM(22, 8, num_layers=3, batch_first=True, dropout=0.1)
  (fc): Linear(in_features=8, out_features=4, bias=True)
)

In [ ]:
print(predictions[0].shape,len(predictions),values[0].shape,len(values))

In [ ]:
values = [y.reshape(-1) for y in values]
predictions = [y.reshape(-1) for y in predictions]

In [ ]:
p_0 = list(zip(*predictions))[0]
p_1 = list(zip(*predictions))[1]
p_2 = list(zip(*predictions))[2]
p_3 = list(zip(*predictions))[3]
print(len(p_0),len(p_1),len(p_2),len(p_3))
print(p_0[0],p_1[0],p_2[0],p_3[0])

In [ ]:
                                                                                                                                                             train_features = torch.Tensor(x_train_all)
train_targets = torch.Tensor(y_train_all)
val_features = torch.Tensor(x_val_all)
val_targets = torch.Tensor(y_val_all)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def calculate_metrics(y, y_hat):
    return {'mae' : mean_absolute_error(df.value, df.prediction),
            'rmse' : mean_squared_error(df.value, df.prediction) ** 0.5,
            'r2' : r2_score(df.value, df.prediction)}

# result_metrics = calculate_metrics(df_result)

In [374]:
def calculate_prediction_results(prediction, ground_truth):
    prediction = np.array(prediction)
    ground_truth = np.array(ground_truth)
    squared_error = np.square(prediction - ground_truth)
    mse = squared_error.mean(axis=None)
    std = squared_error.std(axis=None)
    print(f'{mse} ± {std}')
    return mse, std

In [376]:
calculate_prediction_results(p_0, y_0)
calculate_prediction_results(p_1, y_1)
calculate_prediction_results(p_2, y_2)
calculate_prediction_results(p_3, y_3)

0.01352962239349458 ± 0.013402873554495251
0.786242508900766 ± 2.002830150356002
0.02396993578213799 ± 0.03981948673537908
0.8720320361224081 ± 1.6599422847571033


(0.8720320361224081, 1.6599422847571033)

In [377]:
calculate_prediction_results(b_0, y_0)
calculate_prediction_results(b_1, y_1)
calculate_prediction_results(b_2, y_2)
calculate_prediction_results(b_3, y_3)

0.1064326608673524 ± 0.14163580906447734
5.958845052822763 ± 6.01116957740363
0.04876057121882698 ± 0.07286724208327428
3.3769863060935092 ± 3.5970750169255563


(3.3769863060935092, 3.5970750169255563)

In [ ]:
from sklearn.linear_model import LinearRegression

def build_baseline_model(X_train, y_train, X_test, y_test):

    # X, y = feature_label_split(df, target_col)
    # X_train, X_test, y_train, y_test = train_test_split(
    #     X, y, test_size=test_ratio, shuffle=False
    # )
    result = []
    X_train = np.mean(X_train, axis=1)
    #print(X_train[0].shape)
    y_train = [y.reshape(-1) for y in y_train]
    #print(y_train[0].shape)
    #print(len(X_train),len(y_train))
    X_test = np.mean(X_test, axis=1)
    y_test = [y.reshape(-1) for y in y_test]
    model = LinearRegression()
    #print(y_train[0].shape)
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)

    #print(prediction.shape, type(prediction))

    #result+=prediction
    return prediction

baseline = build_baseline_model(x_train_all, y_train_all, x_val_all, y_val_all)
#baseline_metrics = calculate_metrics(df_baseline)

In [ ]:
print(baseline[0].shape,baseline.shape, type(baseline))

In [ ]:
y_hat = [y.reshape(-1) for y in y_val_all]
y_hat[0].shape

In [ ]:
y_0 = list(zip(*y_hat))[0]
y_1 = list(zip(*y_hat))[1]
y_2 = list(zip(*y_hat))[2]
y_3 = list(zip(*y_hat))[3]
print(len(y_0),len(y_1),len(y_2),len(y_3))
print(y_0[0],y_1[0],y_2[0],y_3[0])

In [ ]:
b_0 = list(zip(*baseline))[0]
b_1 = list(zip(*baseline))[1]
b_2 = list(zip(*baseline))[2]
b_3 = list(zip(*baseline))[3]
print(len(b_0),len(b_1),len(b_2),len(b_3))
print(b_0[0],b_1[0],b_2[0],b_3[0])

In [ ]:
output_column=['position_1', 'velocity_1','position_3', 'velocity_3']

In [367]:
from plotly import graph_objects as go

data = []

value = go.Scatter(
    y=y_0,
    #y=df_result.value,
    mode="lines",
    name="Ground Truth",
    marker=dict(),
   # text=df_result.index,
    line=dict(color="rgba(0,0,0, 0.3)"),
)
data.append(value)

baseline = go.Scatter(
    y=b_0,
   # y=df_baseline.prediction,
    mode="lines",
    line={"dash": "dot"},
    name='Linear Regression Predictions',
    marker=dict(),
    #text=df_baseline.index,
    opacity=0.8,
)

data.append(baseline)

prediction = go.Scatter(
     y=p_0,
     mode="lines",
     line={"dash": "dot"},
     name='LSTM Predictions',
     marker=dict(),
     #text=df_result.index,
     opacity=0.8,
     )
data.append(prediction)

In [358]:
import plotly.offline as pyo

In [368]:
layout = dict(
   # title="Predictions vs Actual Values",
    xaxis=dict(title="Time", ticklen=5, zeroline=False),
    yaxis=dict(title="Right Knee Motor Position", ticklen=5, zeroline=False),
)

fig = dict(data=data, layout=layout)
#a = pyo.iplot(fig)
f = go.Figure(data=data, layout=layout)
print(f)
f.show()

Figure({
    'data': [{'line': {'color': 'rgba(0,0,0, 0.3)'},
              'mode': 'lines',
              'name': 'Ground Truth',
              'type': 'scatter',
              'y': [-0.16569294332887366, -0.17270999583908758,
                    -0.17903153181634995, -0.18418577985678694,
                    -0.16569294332887366, -0.17270999583908758,
                    -0.17903153181634995, -0.18418577985678694,
                    -0.16569294332887366, -0.17270999583908758,
                    -0.17903153181634995, -0.18418577985678694,
                    -0.24987841031194857, -0.2662375869403939, -0.2850009571913125,
                    -0.304456422323713, -0.28975545863210783, -0.29002412518184,
                    -0.2939764936945983, -0.2996667799161248, -0.28975545863210783,
                    -0.29002412518184, -0.2939764936945983, -0.2996667799161248,
                    -0.19148620391640306, -0.18641336441091724,
                    -0.18167863367789525, -0.1802165984004

In [334]:
data = []

value = go.Scatter(
    y=y_1,
    #y=df_result.value,
    mode="lines",
    name="values",
    marker=dict(),
   # text=df_result.index,
    line=dict(color="rgba(0,0,0, 0.3)"),
)
data.append(value)

baseline = go.Scatter(
    y=b_1,
   # y=df_baseline.prediction,
    mode="lines",
    line={"dash": "dot"},
    name='Linear Regression Predictions',
    marker=dict(),
    #text=df_baseline.index,
    opacity=0.8,
)
data.append(baseline)

prediction = go.Scatter(
     y=p_1,
     mode="lines",
     line={"dash": "dot"},
     name='LSTM Predictions',
     marker=dict(),
     #text=df_result.index,
     opacity=0.8,
     )
data.append(prediction)
layout = dict(
    title="Predictions vs Actual Values for the dataset",
    xaxis=dict(title="Time", ticklen=5, zeroline=False),
    yaxis=dict(title="Right Knee Motor Velocity", ticklen=5, zeroline=False),
)

fig = dict(data=data, layout=layout)
pyo.iplot(fig)

In [349]:
type(y_1)
list(y_1)

[-0.26055479735860115,
 -0.25048955747071655,
 -0.19114718577526216,
 -0.1440775034390125,
 -0.26055479735860115,
 -0.25048955747071655,
 -0.19114718577526216,
 -0.1440775034390125,
 -0.26055479735860115,
 -0.25048955747071655,
 -0.19114718577526216,
 -0.1440775034390125,
 -0.49727273364068203,
 -0.6622887640791392,
 -0.7503221406476543,
 -0.7764009379021695,
 0.13432827154237179,
 -0.036715170882026894,
 -0.15949266507095677,
 -0.206270030848638,
 0.13432827154237179,
 -0.036715170882026894,
 -0.15949266507095677,
 -0.206270030848638,
 0.038144679704148726,
 0.2132372658109778,
 0.1953334991729834,
 0.06391230599109501,
 0.038144679704148726,
 0.2132372658109778,
 0.1953334991729834,
 0.06391230599109501,
 0.02921824077989078,
 -0.12523543885434188,
 -0.36821154734291883,
 -0.6704676851186981,
 0.02921824077989078,
 -0.12523543885434188,
 -0.36821154734291883,
 -0.6704676851186981,
 0.02921824077989078,
 -0.12523543885434188,
 -0.36821154734291883,
 -0.6704676851186981,
 -5.0712147715

In [354]:
data_header = ['values']

import csv

with open('velocity_values.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(data_header)
    a = list(y_1)
    write.writerows(a)

Error: iterable expected, not numpy.float64

In [ ]:
data_header = ['Linear Regression Predictions','LSTM Predictions']

import csv

with open('velocity_lr.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(data_header)
    write.writerows(list(b_1))

In [337]:
data_header = ['LSTM Predictions']

import csv

with open('velocity_lstm.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(data_header)
    write.writerows(list(p_1))

Scatter({
    'line': {'color': 'rgba(0,0,0, 0.3)'},
    'mode': 'lines',
    'name': 'values',
    'y': [-0.26055479735860115, -0.25048955747071655, -0.19114718577526216,
          -0.1440775034390125, -0.26055479735860115, -0.25048955747071655,
          -0.19114718577526216, -0.1440775034390125, -0.26055479735860115,
          -0.25048955747071655, -0.19114718577526216, -0.1440775034390125,
          -0.49727273364068203, -0.6622887640791392, -0.7503221406476543,
          -0.7764009379021695, 0.13432827154237179, -0.036715170882026894,
          -0.15949266507095677, -0.206270030848638, 0.13432827154237179,
          -0.036715170882026894, -0.15949266507095677, -0.206270030848638,
          0.038144679704148726, 0.2132372658109778, 0.1953334991729834,
          0.06391230599109501, 0.038144679704148726, 0.2132372658109778,
          0.1953334991729834, 0.06391230599109501, 0.02921824077989078,
          -0.12523543885434188, -0.36821154734291883, -0.6704676851186981,
          0.02

In [ ]:
x_val_all_std = []
for i in x_val_all:
    temp = scalar.transform(i)
    x_val_all_std.append(temp)

In [ ]:
from torch import nn

In [ ]:
seq_model = nn.Sequential(
    nn.Linear(1, 13),
    nn.Tanh(),
    nn.Linear(13, 1))
seq_model

In [ ]:
def training_loop(n_epochs, optimiser, model, loss_fn, X_train,  X_val, y_train, y_val):
    for epoch in range(1, n_epochs + 1):
        output_train = model(X_train) # forwards pass
        loss_train = loss_fn(output_train, y_train) # calculate loss
        output_val = model(X_val)
        loss_val = loss_fn(output_val, y_val)

        optimiser.zero_grad() # set gradients to zero
        loss_train.backward() # backwards pass
        optimiser.step() # update model parameters
        if epoch == 1 or epoch % 10000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [ ]:
optimiser = torch.optim.SGD(seq_model.parameters(), lr=1e-3)
training_loop(
    n_epochs = 200,
    optimiser = optimiser,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    X_train = x_train_all_std,
    X_val = x_val_all_std,
    y_train = y_train_all,
    y_val = y_val_all)

Here is the evolution of a few features over time:

### Inspect and cleanup

Next, look at the statistics of the dataset:

### Normalize the data

It is important to scale features before training a neural network. Normalization is a common way of doing this scaling: subtract the mean and divide by the standard deviation of each feature.

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((x_train_all_std, y_train_all))
val_data = tf.data.Dataset.from_tensor_slices((x_val_all_std, y_val_all))

In [ ]:
train_data

In [ ]:
for image_batch, label_batch in train_data.take(1):
   pass



In [ ]:
print(image_batch.shape, label_batch.shape)

In [ ]:
train_data = train_data.shuffle(24).batch(32)
val_data = val_data.shuffle(24).batch(16)

## BUILD MODELS AND EVALUATE THEM
### Linear model


A `tf.keras.layers.Dense` layer with no `activation` set is a linear model. The layer only transforms the last axis of the data from `(batch, time, inputs)` to `(batch, time, units)`; it is applied independently to every item across the `batch` and `time` axes.

In [ ]:
MAX_EPOCHS = 20

def compile_and_fit(model, train_data, val_data, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  history = model.fit(train_data, epochs=MAX_EPOCHS, batch_size=16,
                      validation_data=val_data, shuffle=True,
                      callbacks=[early_stopping])
  return history

In [ ]:
# multi output
linear_multi = tf.keras.Sequential([
    tf.keras.layers.Dense(units=4)])


In [ ]:
history = compile_and_fit(linear_multi, train_data, val_data)
#performance['Linear'] = linear.evaluate(single_step_window.test, verbose=0)

In [ ]:
val_performance = {}
val_performance['Linear'] = linear_multi.evaluate(val_data)

In [ ]:
plt.bar(x = range(len(train_data.columns)),
        height=linear_multi.layers[0].kernel[:,0].numpy())
axis = plt.gca()
axis.set_xticks(range(len(train_data.columns)))
_ = axis.set_xticklabels(train_data.columns, rotation=90)

### Dense

Before applying models that actually operate on multiple time-steps, it's worth checking the performance of deeper, more powerful, single input step models.

Here's a model similar to the `linear` model, except it stacks several a few `Dense` layers between the input and the output:

In [ ]:
# MULTI OUTPUT
dense = tf.keras.Sequential([
    tf.keras.layers.Dense(units=40, activation='relu'),
    tf.keras.layers.Dense(units=4)
])

history = compile_and_fit(dense, train_data, val_data)

val_performance['Dense'] = dense.evaluate(val_data)

### Multi-step dense

A single-time-step model has no context for the current values of its inputs. It can't see how the input features are changing over time. To address this issue the model needs access to multiple time steps when making predictions:

![Three time steps are used for each prediction.](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/images/conv_window.png?raw=1)


You could train a `dense` model on a multiple-input-step window by adding a `tf.keras.layers.Flatten` as the first layer of the model:

In [ ]:
multi_step_dense_multioutput = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=40, activation='relu'),
    tf.keras.layers.Dense(units=4),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])

In [ ]:
history = compile_and_fit(multi_step_dense_multioutput, train_data,val_data)

IPython.display.clear_output()
val_performance['Multi step dense'] = multi_step_dense_multioutput.evaluate(val_data)

The main down-side of this approach is that the resulting model can only be executed on input windows of exactly this shape.

### Convolution neural network

A convolution layer (`tf.keras.layers.Conv1D`) also takes multiple time steps as input to each prediction.

Below is the **same** model as `multi_step_dense`, re-written with a convolution.

Note the changes:
* The `tf.keras.layers.Flatten` and the first `tf.keras.layers.Dense` are replaced by a `tf.keras.layers.Conv1D`.
* The `tf.keras.layers.Reshape` is no longer necessary since the convolution keeps the time axis in its output.

In [ ]:
CONV_WIDTH = 8
conv_model_multi = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32,
                           kernel_size=(CONV_WIDTH,),
                           activation='relu'),
    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.Dense(units=4),
])

Run it on an example batch to check that the model produces outputs with the expected shape:

In [ ]:
history = compile_and_fit(conv_model_multi, train_data,val_data)

IPython.display.clear_output()
val_performance['Conv'] = conv_model_multi.evaluate(val_data)

The difference between this `conv_model` and the `multi_step_dense` model is that the `conv_model` can be run on inputs of any length. The convolutional layer is applied to a sliding window of inputs:

![Executing a convolutional model on a sequence](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/images/wide_conv_window.png?raw=1)

If you run it on wider input, it produces wider output:

Note that the output is shorter than the input. To make training or plotting work, you need the labels, and prediction to have the same length. So build a `WindowGenerator` to produce wide windows with a few extra input time steps so the label and prediction lengths match:

### Recurrent neural network

A Recurrent Neural Network (RNN) is a type of neural network well-suited to time series data. RNNs process a time series step-by-step, maintaining an internal state from time-step to time-step.

You can learn more in the [Text generation with an RNN](https://www.tensorflow.org/text/tutorials/text_generation) tutorial and the [Recurrent Neural Networks (RNN) with Keras](https://www.tensorflow.org/guide/keras/rnn) guide.

In this tutorial, you will use an RNN layer called Long Short-Term Memory (`tf.keras.layers.LSTM`).

An important constructor argument for all Keras RNN layers, such as `tf.keras.layers.LSTM`, is the `return_sequences` argument. This setting can configure the layer in one of two ways:

1. If `False`, the default, the layer only returns the output of the final time step, giving the model time to warm up its internal state before making a single prediction:

![An LSTM warming up and making a single prediction](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/images/lstm_1_window.png?raw=1)

2. If `True`, the layer returns an output for each input. This is useful for:
  * Stacking RNN layers.
  * Training a model on multiple time steps simultaneously.

![An LSTM making a prediction after every time step](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/images/lstm_many_window.png?raw=1)

In [ ]:
lstm_model_multi = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=4)
])

With `return_sequences=True`, the model can be trained on 24 hours of data at a time.

Note: This will give a pessimistic view of the model's performance. On the first time step, the model has no access to previous steps and, therefore, can't do any better than the simple `linear` and `dense` models shown earlier.

In [ ]:
history = compile_and_fit(lstm_model_multi, train_data,val_data)

IPython.display.clear_output()
val_performance['LSTM'] = lstm_model_multi.evaluate(val_data)
#performance['LSTM'] = lstm_model.evaluate(wide_window.test, verbose=0)

### Performance

With this dataset typically each of the models does slightly better than the one before it:

In [ ]:
x = np.arange(len(val_performance))
width = 0.5
metric_name = 'mean_absolute_error'
metric_index = lstm_model_multi.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in val_performance.values()]
#test_mae = [v[metric_index] for v in performance.values()]

plt.ylabel('mean_absolute_error position_1, normalized]')
plt.bar(x - 0.17, val_mae, width, label='Validation')
#plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=val_performance.keys(), rotation=45)
_ = plt.legend()

In [ ]:
for name, value in val_performance.items():
  print(f'{name:12s}: {value[1]:0.4f}')

### Multi-output models

The models so far all predicted a single output feature, `T (degC)`, for a single time step.

All of these models can be converted to predict multiple features just by changing the number of units in the output layer and adjusting the training windows to include all features in the `labels` (`example_labels`):

In [ ]:
single_step_window = WindowGenerator(
    # `WindowGenerator` returns all features as labels if you
    # don't set the `label_columns` argument.
    input_width=1, label_width=1, shift=1 ,label_columns=['position_1','position_3','velocity_1','velocity_3']
    )

wide_window = WindowGenerator(
    input_width=8, label_width=8, shift=1 ,label_columns=['position_1','position_3','velocity_1','velocity_3']
    )

for example_inputs, example_labels in wide_window.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

In [ ]:
for example_inputs, example_labels in wide_window.val.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

In [ ]:
for example_inputs, example_labels in single_step_window.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

Note above that the `features` axis of the labels now has the same depth as the inputs, instead of `1`.

#### Baseline

The same baseline model (`Baseline`) can be used here, but this time repeating all features instead of selecting a specific `label_index`:

In [ ]:
baseline = Baseline()
baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                 metrics=[tf.keras.metrics.MeanAbsoluteError()])


In [ ]:
val_performance = {}
#performance = {}
val_performance['Baseline'] = baseline.evaluate(wide_window.val)

### trying to build the multioutput single step lstm's baseline here

In [ ]:
single_step_window = WindowGenerator(
    # `WindowGenerator` returns all features as labels if you
    # don't set the `label_columns` argument.
    input_width=1, label_width=1, shift=1 ,label_columns=['position_1','position_3','velocity_1','velocity_3']
    )

#### Dense

In [ ]:
#TESTING THE 4 OUTPUT FUNCTION
FEATURES_NUM = 23
OUTPUT_NUM = 4
dense = tf.keras.Sequential([
    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.Dense(units=OUTPUT_NUM),
    tf.keras.layers.Reshape([1,-1]),
])


In [ ]:
history = compile_and_fit(dense, single_step_window)

IPython.display.clear_output()
val_performance['Dense'] = dense.evaluate(single_step_window.val)

#### RNN


In [ ]:
%%time
num_features = 23
OUTPUT_NUM = 4
wide_window = WindowGenerator(
    input_width=8, label_width=8, shift=1#, label_columns=['position_1','position_3','velocity_1','velocity_3']
    )

lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=num_features)
])

history = compile_and_fit(lstm_model, wide_window)

IPython.display.clear_output()
val_performance['LSTM'] = lstm_model.evaluate( wide_window.val)
#performance['LSTM'] = lstm_model.evaluate( wide_window.test, verbose=0)

print()

In [ ]:
%%time
num_features = 23
OUTPUT_NUM = 4
wide_window = WindowGenerator(
    input_width=8, label_width=8, shift=1, label_columns=['position_1','position_3','velocity_1','velocity_3']
    )

lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=OUTPUT_NUM)
])

history = compile_and_fit(lstm_model, wide_window)

IPython.display.clear_output()
val_performance['LSTM'] = lstm_model.evaluate( wide_window.val)
#performance['LSTM'] = lstm_model.evaluate( wide_window.test, verbose=0)

print()

<a id="residual"></a>

#### Advanced: Residual connections

The `Baseline` model from earlier took advantage of the fact that the sequence doesn't change drastically from time step to time step. Every model trained in this tutorial so far was randomly initialized, and then had to learn that the output is a a small change from the previous time step.

While you can get around this issue with careful initialization, it's  simpler to build this into the model structure.

It's common in time series analysis to build models that instead of predicting the next value, predict how the value will change in the next time step. Similarly, <a href="https://arxiv.org/abs/1512.03385" class="external">residual networks</a>—or ResNets—in deep learning refer to architectures where each layer adds to the model's accumulating result.

That is how you take advantage of the knowledge that the change should be small.

![A model with a residual connection](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/images/residual.png?raw=1)

Essentially, this initializes the model to match the `Baseline`. For this task it helps models converge faster, with slightly better performance.

This approach can be used in conjunction with any model discussed in this tutorial.

Here, it is being applied to the LSTM model, note the use of the `tf.initializers.zeros` to ensure that the initial predicted changes are small, and don't overpower the residual connection. There are no symmetry-breaking concerns for the gradients here, since the `zeros` are only used on the last layer.

In [ ]:
class ResidualWrapper(tf.keras.Model):
  def __init__(self, model):
    super().__init__()
    self.model = model

  def call(self, inputs, *args, **kwargs):
    delta = self.model(inputs, *args, **kwargs)

    # The prediction for each time step is the input
    # from the previous time step plus the delta
    # calculated by the model.
    return inputs + delta

In [ ]:
%%time
residual_lstm = ResidualWrapper(
    tf.keras.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.Dense(
        num_features,
        # The predicted deltas should start small.
        # Therefore, initialize the output layer with zeros.
        kernel_initializer=tf.initializers.zeros()),

]))

history = compile_and_fit(residual_lstm, wide_window)

IPython.display.clear_output()
val_performance['Residual LSTM'] = residual_lstm.evaluate(wide_window.val)
#performance['Residual LSTM'] = residual_lstm.evaluate(wide_window.test, verbose=0)
print()

In [ ]:
%%time
residual_lstm = ResidualWrapper(
    tf.keras.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.Dense(
        num_features,
        # The predicted deltas should start small.
        # Therefore, initialize the output layer with zeros.
        kernel_initializer=tf.initializers.zeros()),

]))

history = compile_and_fit(residual_lstm, wide_window)

IPython.display.clear_output()
val_performance['Residual LSTM'] = residual_lstm.evaluate(wide_window.val)
#performance['Residual LSTM'] = residual_lstm.evaluate(wide_window.test, verbose=0)
print()

#### Performance

Here is the overall performance for these multi-output models.

In [ ]:
x = np.arange(len(val_performance))
width = 0.3

metric_name = 'mean_absolute_error'
metric_index = lstm_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in val_performance.values()]
#test_mae = [v[metric_index] for v in performance.values()]

plt.bar(x - 0.17, val_mae, width, label='Validation')
#plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=val_performance.keys(),
           rotation=45)
plt.ylabel('MAE (average over all outputs)')
_ = plt.legend()

In [ ]:
for name, value in val_performance.items():
  print(f'{name:15s}: {value[1]:0.4f}')

In [ ]:
def plot(self, model=None, plot_col='position_1', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_n, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time')


The above performances are averaged across all model outputs.

## Multi-step models

Both the single-output and multiple-output models in the previous sections made **single time step predictions**, one hour into the future.

This section looks at how to expand these models to make **multiple time step predictions**.

In a multi-step prediction, the model needs to learn to predict a range of future values. Thus, unlike a single step model, where only a single future point is predicted, a multi-step model predicts a sequence of the future values.

There are two rough approaches to this:

1. Single shot predictions where the entire time series is predicted at once.
2. Autoregressive predictions where the model only makes single step predictions and its output is fed back as its input.

In this section all the models will predict **all the features across all output time steps**.


For the multi-step model, the training data again consists of hourly samples. However, here, the models will learn to predict 24 hours into the future, given 24 hours of the past.

Here is a `Window` object that generates these slices from the dataset:

In [ ]:
OUT_STEPS = 24
multi_window = WindowGenerator(input_width=24,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS)

multi_window.plot()
multi_window

### Baselines

A simple baseline for this task is to repeat the last input time step for the required number of output time steps:

![Repeat the last input, for each output step](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/images/multistep_last.png?raw=1)

In [ ]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    return tf.tile(inputs[:, -1:, :], [1, OUT_STEPS, 1])

last_baseline = MultiStepLastBaseline()
last_baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                      metrics=[tf.keras.metrics.MeanAbsoluteError()])

multi_val_performance = {}
multi_performance = {}

multi_val_performance['Last'] = last_baseline.evaluate(multi_window.val)
multi_performance['Last'] = last_baseline.evaluate(multi_window.test, verbose=0)
multi_window.plot(last_baseline)

Since this task is to predict 24 hours into the future, given 24 hours of the past, another simple approach is to repeat the previous day, assuming tomorrow will be similar:

![Repeat the previous day](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/images/multistep_repeat.png?raw=1)

In [ ]:
class RepeatBaseline(tf.keras.Model):
  def call(self, inputs):
    return inputs

repeat_baseline = RepeatBaseline()
repeat_baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                        metrics=[tf.keras.metrics.MeanAbsoluteError()])

multi_val_performance['Repeat'] = repeat_baseline.evaluate(multi_window.val)
multi_performance['Repeat'] = repeat_baseline.evaluate(multi_window.test, verbose=0)
multi_window.plot(repeat_baseline)

### Single-shot models

One high-level approach to this problem is to use a "single-shot" model, where the model makes the entire sequence prediction in a single step.

This can be implemented efficiently as a `tf.keras.layers.Dense` with `OUT_STEPS*features` output units. The model just needs to reshape that output to the required `(OUTPUT_STEPS, features)`.

#### Linear

A simple linear model based on the last input time step does better than either baseline, but is underpowered. The model needs to predict `OUTPUT_STEPS` time steps, from a single input time step with a linear projection. It can only capture a low-dimensional slice of the behavior, likely based mainly on the time of day and time of year.

![Predict all timesteps from the last time-step](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/images/multistep_dense.png?raw=1)

In [ ]:
multi_linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_linear_model, multi_window)

IPython.display.clear_output()
multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val)
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_linear_model)

#### Dense

Adding a `tf.keras.layers.Dense` between the input and output gives the linear model more power, but is still only based on a single input time step.

#### CNN

A convolutional model makes predictions based on a fixed-width history, which may lead to better performance than the dense model since it can see how things are changing over time:

![A convolutional model sees how things change over time](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/images/multistep_conv.png?raw=1)

#### RNN

A recurrent model can learn to use a long history of inputs, if it's relevant to the predictions the model is making. Here the model will accumulate internal state for 24 hours, before making a single prediction for the next 24 hours.

In this single-shot format, the LSTM only needs to produce an output at the last time step, so set `return_sequences=False` in `tf.keras.layers.LSTM`.

![The LSTM accumulates state over the input window, and makes a single prediction for the next 24 hours](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/images/multistep_lstm.png?raw=1)


In [ ]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.LSTM(32, return_sequences=False),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_lstm_model, multi_window)

IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_lstm_model)

### Advanced: Autoregressive model

The above models all predict the entire output sequence in a single step.

In some cases it may be helpful for the model to decompose this prediction into individual time steps. Then, each model's output can be fed back into itself at each step and predictions can be made conditioned on the previous one, like in the classic <a href="https://arxiv.org/abs/1308.0850" class="external">Generating Sequences With Recurrent Neural Networks</a>.

One clear advantage to this style of model is that it can be set up to produce output with a varying length.

You could take any of the single-step multi-output models trained in the first half of this tutorial and run in an autoregressive feedback loop, but here you'll focus on building a model that's been explicitly trained to do that.

![Feedback a model's output to its input](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/images/multistep_autoregressive.png?raw=1)

#### RNN

This tutorial only builds an autoregressive RNN model, but this pattern could be applied to any model that was designed to output a single time step.

The model will have the same basic form as the single-step LSTM models from earlier: a `tf.keras.layers.LSTM` layer followed by a `tf.keras.layers.Dense` layer that converts the `LSTM` layer's outputs to model predictions.

A `tf.keras.layers.LSTM` is a `tf.keras.layers.LSTMCell` wrapped in the higher level `tf.keras.layers.RNN` that manages the state and sequence results for you (Check out the [Recurrent Neural Networks (RNN) with Keras](https://www.tensorflow.org/guide/keras/rnn) guide for details).

In this case, the model has to manually manage the inputs for each step, so it uses `tf.keras.layers.LSTMCell` directly for the lower level, single time step interface.

In [ ]:
class FeedBack(tf.keras.Model):
  def __init__(self, units, out_steps):
    super().__init__()
    self.out_steps = out_steps
    self.units = units
    self.lstm_cell = tf.keras.layers.LSTMCell(units)
    # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
    self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
    self.dense = tf.keras.layers.Dense(num_features)

In [ ]:
feedback_model = FeedBack(units=32, out_steps=OUT_STEPS)

The first method this model needs is a `warmup` method to initialize its internal state based on the inputs. Once trained, this state will capture the relevant parts of the input history. This is equivalent to the single-step `LSTM` model from earlier:

In [ ]:
def warmup(self, inputs):
  # inputs.shape => (batch, time, features)
  # x.shape => (batch, lstm_units)
  x, *state = self.lstm_rnn(inputs)

  # predictions.shape => (batch, features)
  prediction = self.dense(x)
  return prediction, state

FeedBack.warmup = warmup

This method returns a single time-step prediction and the internal state of the `LSTM`:

In [ ]:
prediction, state = feedback_model.warmup(multi_window.example[0])
prediction.shape
